In [5]:
# !pip install timm flash_attn

In [1]:
import requests
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM 
import pandas as pd
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
from transformers import (AdamW, AutoProcessor, get_scheduler)
from tqdm import tqdm

In [2]:

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-base-ft", trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base-ft", trust_remote_code=True)


cuda:0


config.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


processing_florence2.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/home/vishwa/Enter/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
def run_example(task_prompt, text_prompt, image):
    # prompt = task_prompt + text_input
    image = Image.open(image)
    inputs = processor(text=text_prompt, images=image, return_tensors="pt").to(device, torch_dtype)
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        num_beams=3
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(generated_text, task=task_prompt, image_size=(image.width, image.height))
    return parsed_answer

In [4]:
class FinetuneDataset(Dataset):
    def __init__(self, data, mode):
        self.data = data
        self.mode = mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data.loc[idx]
        # print(example)
        task_prompt = "<MORE_DETAILED_CAPTION>" 
        text_prompt = example['entity_name']
        answer = example['entity_value']
        image_path = '/home/vishwa/amzn-ml/student_resource 3/train_images/' + example['image_link']
        image = Image.open(image_path)
        # image = Image.open(image)
        # if image.mode != "RGB":
            # image = image.convert("RGB")
        return task_prompt, text_prompt, answer, image

In [5]:
# %%time
# task_prompt = "<MORE_DETAILED_CAPTION>" 
# text_prompt = "numbers"
# # url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
# image = Image.open('/home/vishwa/amzn-ml/student_resource 3/images/41uwo4PVnuL.jpg')
# inputs = processor(text=text_prompt, images=image, return_tensors="pt").to(device, torch_dtype)

# generated_ids = model.generate(
#     input_ids=inputs["input_ids"],
#     pixel_values=inputs["pixel_values"],
#     max_new_tokens=1024,
#     num_beams=3,
#     do_sample=False
# )

# generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

# parsed_answer = processor.post_process_generation(generated_text, task=task_prompt, image_size=(image.width, image.height))

# print(parsed_answer)

# Figuring out the valid & train set

In [6]:
train_df = pd.read_csv('student_resource 3/dataset/train.csv')
test_df = pd.read_csv('student_resource 3/dataset/test.csv')

In [7]:
train_groups = set(train_df['group_id'].unique())
print('Train group size:', len(train_groups))
test_groups = set(test_df['group_id'].unique())
print('Test group size:', len(test_groups))
overlap = list(train_groups.intersection(test_groups))
print('Overlap size:', len(overlap))

Train group size: 750
Test group size: 924
Overlap size: 638


In [8]:
len(train_groups - set(overlap))

112

## Since the test set has some groups that are not in the train set, my approach is to work 

In [9]:
def split():
    return  

In [10]:
train_df['train/valid'] = np.where(train_df['group_id'].isin(overlap), 'train', 'valid')

In [11]:
train_df['image_link'] = train_df['image_link'].str.split('/').str[-1]

In [12]:
train_dat = train_df[train_df['train/valid'] == 'train']
valid_dat = train_df[train_df['train/valid'] == 'valid']

In [13]:
train_dat.reset_index(inplace=True)
train_dat

,index,image_link,group_id,entity_name,entity_value,train/valid
0,0,61I9XdN6OFL.jpg,748919,item_weight,500.0 gram,train
1,1,71gSRbyXmoL.jpg,916768,item_volume,1.0 cup,train
2,2,61BZ4zrjZXL.jpg,459516,item_weight,0.709 gram,train
3,3,612mrlqiI4L.jpg,459516,item_weight,0.709 gram,train
4,4,617Tl40LOXL.jpg,731432,item_weight,1400 milligram,train
...,...,...,...,...,...,...
260054,263854,612J1R1xHlL.jpg,558806,height,5.0 centimetre,train
260055,263855,61Blzh2+28L.jpg,470067,height,8.5 inch,train
260056,263856,51MsegDL9VL.jpg,204245,height,43.2 centimetre,train
260057,263857,510KhVw4VSL.jpg,752266,height,9.1 centimetre,train


In [14]:
valid_dat.reset_index(inplace=True)
valid_dat

,index,image_link,group_id,entity_name,entity_value,train/valid
0,7196,51UnMWmk+WL.jpg,793731,item_weight,450.0 gram,valid
1,16238,61MlmQnMzBL.jpg,793731,item_weight,101.0 gram,valid
2,16373,71PswiXNbcL.jpg,793731,item_weight,400.0 gram,valid
3,25580,61x5eHpmrNL.jpg,793731,item_weight,150.0 gram,valid
4,26858,71Iub8eRH7L.jpg,793731,item_weight,3.0 ounce,valid
...,...,...,...,...,...,...
3795,263598,41Lp7a25kYL.jpg,108478,height,15.3 inch,valid
3796,263693,612nLFhf3kL.jpg,318766,height,91.0 centimetre,valid
3797,263700,51nCNRn+fcL.jpg,901135,height,24.0 centimetre,valid
3798,263777,51qAAQELGLL.jpg,251430,height,18.0 centimetre,valid


# Finetuning code

In [15]:
train_dataset = FinetuneDataset(train_dat,'train')
next(iter(train_dataset))



('<MORE_DETAILED_CAPTION>',
 'item_weight',
 '500.0 gram',
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1600x1600>)

In [16]:
valid_dataset = FinetuneDataset(valid_dat,'valid')
next(iter(valid_dataset))

('<MORE_DETAILED_CAPTION>',
 'item_weight',
 '450.0 gram',
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1010x1010>)

In [17]:
def collate_fn(batch):
    task_prompt, text_prompt, answers, images = zip(*batch)
    inputs = processor(text=list(text_prompt), images=list(images), return_tensors="pt", padding=True).to(device)
    return inputs, answers
batch_size = 1
num_workers = 0
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=num_workers)
val_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)


In [20]:
def train_model(train_loader, val_loader, model, processor, epochs=11, lr=1e-6):
    optimizer = AdamW(model.parameters(), lr=lr)
    num_training_steps = epochs * len(train_loader)
    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        i = -1
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{epochs}"):
            i += 1
            inputs, answers = batch

            input_ids = inputs["input_ids"]
            pixel_values = inputs["pixel_values"]
            labels = processor.tokenizer(text=answers, return_tensors="pt", padding=True, return_token_type_ids=False).input_ids.to(device)
            outputs = model(input_ids=input_ids, pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        print(f"Average Training Loss: {avg_train_loss}")

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}/{epochs}"):
                inputs, answers = batch

                input_ids = inputs["input_ids"]
                pixel_values = inputs["pixel_values"]
                labels = processor.tokenizer(text=answers, return_tensors="pt", padding=True, return_token_type_ids=False).input_ids.to(device)

                outputs = model(input_ids=input_ids, pixel_values=pixel_values, labels=labels)
                loss = outputs.loss

                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        print(f"Average Validation Loss: {avg_val_loss}")

        # Save model checkpoint
        output_dir = f"./model_checkpoints/epoch_{epoch+1}"
        os.makedirs(output_dir, exist_ok=True)
        model.save_pretrained(output_dir)
        processor.save_pretrained(output_dir)


In [22]:
     
# Note: We will freeze image encoder for this tutorial. The authors have reported improvement in unfreezing image encoder, but note that this will result in more resource usage.
for param in model.vision_tower.parameters():
  param.is_trainable = False
  
train_model(train_loader, val_loader, model, processor, epochs=1)


Training Epoch 1/1:   0%|          | 6/260059 [00:43<527:38:19,  7.30s/it]


KeyboardInterrupt: 